In [1]:
# jupyter imports
from IPython.display import clear_output


In [2]:
import pandas as pd

In [3]:
FILE_TO_ADD_PATH =  '/sci/archive/michall/roeizucker/IIH/GWAS_standart/results/updated_Both.csv'
FILE_TO_CREATE_PATH = '/sci/archive/michall/roeizucker/IIH/GWAS_standart/results/updated_Both_with_genes.csv'
GENE_FILE_PATH = '/cs/labs/michall/roeizucker/virt_env_install_test/genes_hg19.csv'
WORK_ONLY_FOR_RES_VARIANTS = True

In [4]:
CHR_SYMBOL = "#CHROM"
POS_SYMBOL = "POS"
PVAL_SYMBOL = "P"

In [5]:
def find_gene_heler(tree,row):
    if tree is None:
        return None
    if row["POS"] <= tree.val["cds_end"] and row["POS"] >= tree.val["cds_start"]:
        return tree.val
    if row["POS"] <= tree.val["cds_start"]:
        return find_gene_heler(tree.left,row)
    return find_gene_heler(tree.right,row)
    
def find_gene(genes_dict,row):
    return find_gene_heler((genes_dict[str(row["#CHROM"])]),row)
    pass

def add_genes_for_snps_symbol(df,genes_csv):
    rows = []
    for index,row in df.iterrows():
        if index % 10000 == 0:
            clear_output(wait=True)
            print(str(int(index/len(df) * 100)) + "%")

        val = row
        gene = find_gene(genes_dict,row)
        rows.append([row,gene])
    return rows
    

In [6]:
# create gene tree
class TreeNode:
    def __init__(self,val=None,left=None,right=None):
        self.val = val
        self.left = left
        self.right = right

def add_to_node(node,val):
    if val["cds_start"] < node.val["cds_start"]:
        if node.left is None:
            node.left = TreeNode(val)
        else:
            add_to_node(node.left,val)
    else:
        if node.right is None:
            node.right = TreeNode(val)
        else:
            add_to_node(node.right,val)
    pass
genes_df = pd.read_csv(GENE_FILE_PATH)
genes_dict = {}
for index, row in genes_df.iterrows():
    if row['chr'] not in genes_dict:
        genes_dict[row['chr']] = TreeNode(row)
    else:
        add_to_node(genes_dict[row['chr']],row)



In [7]:
# genes_dict

In [8]:
# print(find_gene(genes_dict,first_row))

In [9]:
# test_csv = "/cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_small_cov.csv"
# test_df = pd.read_csv(test_csv)
# first_row = None
# for val in test_df.iterrows():
#     first_row = val[1]
#     if find_gene(genes_dict,first_row) is not None:
#         print(find_gene(genes_dict,first_row) ,val)
#         print("#"*30)
    
# first_row

In [10]:
# add_genes_for_snps_symbol(df,genes_df)


In [11]:
df = pd.read_csv(FILE_TO_ADD_PATH)
# df["length"] = df["length"].astype(int)
# df.to_csv(FILE_TO_CREATE_PATH)

In [12]:
if WORK_ONLY_FOR_RES_VARIANTS:
    df = df[df[PVAL_SYMBOL].notnull()]
vals = add_genes_for_snps_symbol(df,genes_df)


99%


In [13]:
index = 0
final_results = []
for val in vals:
    if index % 10000 == 0:
        clear_output(wait=True)
        print(str(int(index/len(df) * 100)) + "%")
    index+=1    
    new_0 = val[0].to_dict()
    if val[1] is None:
#         val[0]["gene_symbol"] = "no_gene_found"
#         val[0]["length"] = -1
#         val[0]["name"] = "no_gene_found"
#         final_results.append(val[0])
        new_0["gene_symbol"] = "no_gene_found"
        new_0["length"] = -1
        new_0["name"] = "no_gene_found"
        final_results.append(new_0)

        continue
    new_1 = val[1].to_dict()
    new_0["gene_symbol"] = new_1["symbol"]
    new_0["length"] =  int(new_1["cds_end"]) -  int(new_1["cds_start"])
    new_0["name"] = new_1["name"]
    final_results.append(new_0)
    

99%


In [14]:
len(final_results)

10258503

In [15]:
new_df = pd.DataFrame.from_records(final_results)

In [16]:
new_df.to_csv(FILE_TO_CREATE_PATH)

In [17]:
new_df[new_df["length"] > 0]

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE,gene_symbol,length,name
513,1,861920,rs537774543,C,T,Y,T,C,0.001784,Y,ADD,284223,0.812832,1.399610,-0.148063,0.882293,.,SAMD11,18208,sterile alpha motif domain containing 11
514,1,862772,rs192998324,A,G,Y,G,A,0.030979,N,ADD,291312,0.834015,0.337744,-0.537402,0.590990,.,SAMD11,18208,sterile alpha motif domain containing 11
515,1,863360,rs568926072,C,G,Y,G,C,0.005664,N,ADD,289567,0.518018,1.002000,-0.656430,0.511547,.,SAMD11,18208,sterile alpha motif domain containing 11
516,1,863499,rs28718350,T,C,Y,C,T,0.032279,N,ADD,273519,0.931639,0.321252,-0.220420,0.825544,.,SAMD11,18208,sterile alpha motif domain containing 11
517,1,863538,rs570731716,C,T,Y,T,C,0.003486,N,ADD,287618,0.853697,1.001860,-0.157886,0.874547,.,SAMD11,18208,sterile alpha motif domain containing 11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258468,22,51219704,rs147475742,G,A,Y,A,G,0.037112,N,ADD,284541,1.172690,0.264241,0.602848,0.546610,.,RABL2B,13515,"RAB, member of RAS oncogene family like 2B"
10258469,22,51220088,rs566371895,G,A,Y,A,G,0.002286,Y,ADD,289865,0.622804,1.400760,-0.338047,0.735328,.,RABL2B,13515,"RAB, member of RAS oncogene family like 2B"
10258470,22,51220249,rs9616977,A,T,Y,T,A,0.071194,N,ADD,290129,0.968158,0.211969,-0.152663,0.878664,.,RABL2B,13515,"RAB, member of RAS oncogene family like 2B"
10258471,22,51220319,rs9616978,C,G,Y,G,C,0.067436,N,ADD,288594,1.025820,0.212073,0.120207,0.904319,.,RABL2B,13515,"RAB, member of RAS oncogene family like 2B"


In [18]:
df[df["P"] < 0.00005]

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
10509,1,3363272,rs148984078,G,A,Y,A,G,0.008083,N,ADD,285614,3.70141,0.319938,4.09052,4.304030e-05,.
12345,1,3756568,rs530230016,C,T,Y,T,C,0.001346,N,ADD,290768,8.36159,0.497022,4.27275,1.930770e-05,.
23460,1,6095424,rs537531827,T,C,Y,C,T,0.001796,N,ADD,290889,8.32086,0.453063,4.67654,2.917570e-06,.
27210,1,7090989,rs537339895,G,T,Y,T,G,0.000861,N,ADD,290854,13.91830,0.508347,5.17993,2.219670e-07,.
36230,1,9543230,rs116488611,G,A,Y,A,G,0.001861,N,ADD,289605,8.07715,0.455196,4.58932,4.446980e-06,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10240829,22,47893772,rs151017429,C,T,Y,T,C,0.004659,N,ADD,290702,5.02001,0.357877,4.50834,6.533710e-06,.
10241718,22,48074516,rs192540303,G,A,Y,A,G,0.004197,N,ADD,289386,4.92025,0.383802,4.15152,3.302790e-05,.
10245023,22,48729388,rs138188538,G,C,Y,C,G,0.004087,N,ADD,290221,5.71436,0.354693,4.91406,8.921070e-07,.
10252501,22,49880793,rs541764107,A,G,Y,G,A,0.001255,N,ADD,290398,9.40977,0.507775,4.41485,1.010820e-05,.


In [19]:
# df = df.drop("GENE",axis='columns')
# df.to_csv(FILE_TO_CREATE_PATH)

In [20]:
genes_df[(genes_df["chr"] == "1") & (genes_df["cds_start"] < 11253684) & (genes_df["cds_end"] > 11253684)]

,Unnamed: 0,uniprot_id,symbol,name,refseq_ids,chr,cds_start,cds_end
1467,1467,O43827,ANGPTL7,angiopoietin like 7,['NM_021146'],1,11249637,11255077
4715,4715,P42345,MTOR,mechanistic target of rapamycin kinase,['NM_004958'],1,11167545,11319466


In [21]:
df

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
0,1,10616,1:10616_CCGCCGTTGCAAAGGCGCGCCG_C,CCGCCGTTGCAAAGGCGCGCCG,C,Y,CCGCCGTTGCAAAGGCGCGCCG,C,0.001399,N,ADD,285564,4.284630,0.709767,2.050020,0.040363,.
1,1,13453,rs568927457,T,C,Y,C,T,0.001221,Y,ADD,282252,1.246500,1.400320,0.157350,0.874969,.
3,1,15585,rs533630043,G,A,Y,A,G,0.001966,Y,ADD,283783,0.752308,1.399980,-0.203296,0.838904,.
4,1,15774,rs374029747,G,A,Y,A,G,0.000896,Y,ADD,280246,1.665460,1.385690,0.368120,0.712784,.
5,1,15777,rs568149713,A,G,Y,G,A,0.003444,N,ADD,271335,0.861714,1.002550,-0.148453,0.881985,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258622,22,51239296,rs8137179,T,C,Y,C,T,0.000878,N,ADD,290556,6.682160,0.712845,2.664590,0.007708,.
10258623,22,51239304,rs8142977,C,T,Y,T,C,0.000878,N,ADD,290556,6.682160,0.712845,2.664590,0.007708,.
10258624,22,51239586,rs535432390,T,G,Y,G,T,0.001024,N,ADD,289420,8.429550,0.584121,3.649490,0.000263,.
10258625,22,51241386,rs568168135,C,G,Y,G,C,0.002572,N,ADD,284411,2.245790,0.710336,1.138980,0.254711,.
